In [39]:
import re
import numpy as np
import pandas as pd
import seaborn as sns
import nltk
from bs4 import BeautifulSoup
%matplotlib inline
import matplotlib.pyplot as plt

# Vanilla RNN

![rnn](https://i.imgur.com/tsul5TY.png)

* Conventional RNN
* Usually with sigmoid activation
* is trained upon unfonding k-times using the BPTTalgorithm

Disadvantages
* upon multiple nestings sigmoid(-like) functions lead to exponential decay of error information
* vanishing gradient problem
* exploding gradient problem
* too simple architecture
* not flexible enoughfor both short and long-term dependencies
* too simplistic however still computationallyexpensive

# Natural Language Processing

Dataset \
https://www.kaggle.com/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews

In [129]:
df = pd.read_csv('./datasets./IMDB Dataset.csv')
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [130]:
print(df.review[2])

I thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. The plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). While some may be disappointed when they realize this is not Match Point 2: Risk Addiction, I thought it was proof that Woody Allen is still fully in control of the style many of us have grown to love.<br /><br />This was the most I'd laughed at one of Woody's comedies in years (dare I say a decade?). While I've never been impressed with Scarlet Johanson, in this she managed to tone down her "sexy" image and jumped right into a average, but spirited young woman.<br /><br />This may not be the crown jewel of his career, but it was wittier than "Devil Wears Prada" and more interesting than "Superman" a great comedy to go see with friends.


## Text normalization

Reduction of randomness in text. Bringing it closer to a predefined standard. Words can be written in multiple ways or mispelled, especially in social media.

### Cleaning

**Lowercasing all the words**

In [131]:
df.review = df.review.apply(lambda s: s.lower())
print(df.review[2])

i thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. the plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). while some may be disappointed when they realize this is not match point 2: risk addiction, i thought it was proof that woody allen is still fully in control of the style many of us have grown to love.<br /><br />this was the most i'd laughed at one of woody's comedies in years (dare i say a decade?). while i've never been impressed with scarlet johanson, in this she managed to tone down her "sexy" image and jumped right into a average, but spirited young woman.<br /><br />this may not be the crown jewel of his career, but it was wittier than "devil wears prada" and more interesting than "superman" a great comedy to go see with friends.


**Cleaning noise like html tags and punctuation**

In [132]:
df.review = df.review.apply(lambda s: BeautifulSoup(s).get_text())
print(df.review[2])

i thought this was a wonderful way to spend time on a too hot summer weekend, sitting in the air conditioned theater and watching a light-hearted comedy. the plot is simplistic, but the dialogue is witty and the characters are likable (even the well bread suspected serial killer). while some may be disappointed when they realize this is not match point 2: risk addiction, i thought it was proof that woody allen is still fully in control of the style many of us have grown to love.this was the most i'd laughed at one of woody's comedies in years (dare i say a decade?). while i've never been impressed with scarlet johanson, in this she managed to tone down her "sexy" image and jumped right into a average, but spirited young woman.this may not be the crown jewel of his career, but it was wittier than "devil wears prada" and more interesting than "superman" a great comedy to go see with friends.


In [133]:
pattern = r'[^a-zA-z0-9\s]'

df.review = df.review.apply(lambda s: re.sub(pattern, '', s))
print(df.review[2])

i thought this was a wonderful way to spend time on a too hot summer weekend sitting in the air conditioned theater and watching a lighthearted comedy the plot is simplistic but the dialogue is witty and the characters are likable even the well bread suspected serial killer while some may be disappointed when they realize this is not match point 2 risk addiction i thought it was proof that woody allen is still fully in control of the style many of us have grown to lovethis was the most id laughed at one of woodys comedies in years dare i say a decade while ive never been impressed with scarlet johanson in this she managed to tone down her sexy image and jumped right into a average but spirited young womanthis may not be the crown jewel of his career but it was wittier than devil wears prada and more interesting than superman a great comedy to go see with friends


### Tokenize

In [134]:
nltk.download('punkt')

df.review = df.review.apply(lambda s: nltk.word_tokenize(s))
print(df.review[2])

[nltk_data] Downloading package punkt to C:\Users\Konrad
[nltk_data]     Ulman\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['i', 'thought', 'this', 'was', 'a', 'wonderful', 'way', 'to', 'spend', 'time', 'on', 'a', 'too', 'hot', 'summer', 'weekend', 'sitting', 'in', 'the', 'air', 'conditioned', 'theater', 'and', 'watching', 'a', 'lighthearted', 'comedy', 'the', 'plot', 'is', 'simplistic', 'but', 'the', 'dialogue', 'is', 'witty', 'and', 'the', 'characters', 'are', 'likable', 'even', 'the', 'well', 'bread', 'suspected', 'serial', 'killer', 'while', 'some', 'may', 'be', 'disappointed', 'when', 'they', 'realize', 'this', 'is', 'not', 'match', 'point', '2', 'risk', 'addiction', 'i', 'thought', 'it', 'was', 'proof', 'that', 'woody', 'allen', 'is', 'still', 'fully', 'in', 'control', 'of', 'the', 'style', 'many', 'of', 'us', 'have', 'grown', 'to', 'lovethis', 'was', 'the', 'most', 'id', 'laughed', 'at', 'one', 'of', 'woodys', 'comedies', 'in', 'years', 'dare', 'i', 'say', 'a', 'decade', 'while', 'ive', 'never', 'been', 'impressed', 'with', 'scarlet', 'johanson', 'in', 'this', 'she', 'managed', 'to', 'tone', 'down',

### Stop word removal

Stop words in english are low information words like "a, is, are" etc.

In [135]:
nltk.download('stopwords')

stopwords = nltk.corpus.stopwords.words('english')
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to C:\Users\Konrad
[nltk_data]     Ulman\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [136]:
df.review = df.review.apply(lambda s: [w for w in s if w not in stopwords])
print(df.review[2])

['thought', 'wonderful', 'way', 'spend', 'time', 'hot', 'summer', 'weekend', 'sitting', 'air', 'conditioned', 'theater', 'watching', 'lighthearted', 'comedy', 'plot', 'simplistic', 'dialogue', 'witty', 'characters', 'likable', 'even', 'well', 'bread', 'suspected', 'serial', 'killer', 'may', 'disappointed', 'realize', 'match', 'point', '2', 'risk', 'addiction', 'thought', 'proof', 'woody', 'allen', 'still', 'fully', 'control', 'style', 'many', 'us', 'grown', 'lovethis', 'id', 'laughed', 'one', 'woodys', 'comedies', 'years', 'dare', 'say', 'decade', 'ive', 'never', 'impressed', 'scarlet', 'johanson', 'managed', 'tone', 'sexy', 'image', 'jumped', 'right', 'average', 'spirited', 'young', 'womanthis', 'may', 'crown', 'jewel', 'career', 'wittier', 'devil', 'wears', 'prada', 'interesting', 'superman', 'great', 'comedy', 'go', 'see', 'friends']


### Stemming

Reducing words to their root or stem \

Example:
* connect -> connect
* connected -> connect
* connects -> connect


* trouble -> troubl
* troubles -> troubl

In [144]:
stemmer = nltk.stem.SnowballStemmer('english')

df['review_stem'] = df.review.apply(lambda s: [stemmer.stem(w) for w in s])
print(df.review_stem[2])

['thought', 'wonder', 'way', 'spend', 'time', 'hot', 'summer', 'weekend', 'sit', 'air', 'condit', 'theater', 'watch', 'lightheart', 'comedi', 'plot', 'simplist', 'dialogu', 'witti', 'charact', 'likabl', 'even', 'well', 'bread', 'suspect', 'serial', 'killer', 'may', 'disappoint', 'realiz', 'match', 'point', '2', 'risk', 'addict', 'thought', 'proof', 'woodi', 'allen', 'still', 'fulli', 'control', 'style', 'mani', 'us', 'grown', 'lovethi', 'id', 'laugh', 'one', 'woodi', 'comedi', 'year', 'dare', 'say', 'decad', 'ive', 'never', 'impress', 'scarlet', 'johanson', 'manag', 'tone', 'sexi', 'imag', 'jump', 'right', 'averag', 'spirit', 'young', 'womanthi', 'may', 'crown', 'jewel', 'career', 'wittier', 'devil', 'wear', 'prada', 'interest', 'superman', 'great', 'comedi', 'go', 'see', 'friend']


### Lemmatization

Transform words to their root form

Example:
* connect -> connect
* connected -> connect
* connects -> connect


* trouble -> trouble
* troubles -> trouble

In [145]:
nltk.download('wordnet')
nltk.download('omw-1.4')
lemmatizer = nltk.stem.WordNetLemmatizer()

df['review_lemm'] = df.review.apply(lambda s: [lemmatizer.lemmatize(w) for w in s])
print(df.review_lemm[2])

['thought', 'wonderful', 'way', 'spend', 'time', 'hot', 'summer', 'weekend', 'sitting', 'air', 'conditioned', 'theater', 'watching', 'lighthearted', 'comedy', 'plot', 'simplistic', 'dialogue', 'witty', 'character', 'likable', 'even', 'well', 'bread', 'suspected', 'serial', 'killer', 'may', 'disappointed', 'realize', 'match', 'point', '2', 'risk', 'addiction', 'thought', 'proof', 'woody', 'allen', 'still', 'fully', 'control', 'style', 'many', 'u', 'grown', 'lovethis', 'id', 'laughed', 'one', 'woodys', 'comedy', 'year', 'dare', 'say', 'decade', 'ive', 'never', 'impressed', 'scarlet', 'johanson', 'managed', 'tone', 'sexy', 'image', 'jumped', 'right', 'average', 'spirited', 'young', 'womanthis', 'may', 'crown', 'jewel', 'career', 'wittier', 'devil', 'wear', 'prada', 'interesting', 'superman', 'great', 'comedy', 'go', 'see', 'friend']


### Tagging

Enriching data with information about words, like tagging words with the part of speech. Example whether book is a verb or a noun.

In [151]:
nltk.download('tagsets')
print(nltk.help.upenn_tagset())

$: dollar
    $ -$ --$ A$ C$ HK$ M$ NZ$ S$ U.S.$ US$
'': closing quotation mark
    ' ''
(: opening parenthesis
    ( [ {
): closing parenthesis
    ) ] }
,: comma
    ,
--: dash
    --
.: sentence terminator
    . ! ?
:: colon or ellipsis
    : ; ...
CC: conjunction, coordinating
    & 'n and both but either et for less minus neither nor or plus so
    therefore times v. versus vs. whether yet
CD: numeral, cardinal
    mid-1890 nine-thirty forty-two one-tenth ten million 0.5 one forty-
    seven 1987 twenty '79 zero two 78-degrees eighty-four IX '60s .025
    fifteen 271,124 dozen quintillion DM2,000 ...
DT: determiner
    all an another any both del each either every half la many much nary
    neither no some such that the them these this those
EX: existential there
    there
FW: foreign word
    gemeinschaft hund ich jeux habeas Haementeria Herr K'ang-si vous
    lutihaw alai je jour objets salutaris fille quibusdam pas trop Monte
    terram fiche oui corporis ...
IN: preposition or

[nltk_data] Downloading package tagsets to C:\Users\Konrad
[nltk_data]     Ulman\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!


In [147]:
nltk.download('averaged_perceptron_tagger')

df['tags'] = df.review.apply(lambda s: nltk.pos_tag(s))
df.head()

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Konrad Ulman\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


0    [(one, CD), (reviewers, NNS), (mentioned, VBD)...
1    [(wonderful, JJ), (little, JJ), (production, N...
2    [(thought, VBN), (wonderful, JJ), (way, NN), (...
3    [(basically, RB), (theres, NNS), (family, NN),...
4    [(petter, NN), (matteis, RBS), (love, JJ), (ti...
Name: tags, dtype: object

# Text classification

# Time function

# Generating text

# Translating text